<a href="https://colab.research.google.com/github/ksyeon94/untitled/blob/main/ner_label_chaged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install transformers
!pip -q install datasets
!pip -q install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from pathlib import Path
from pprint import pprint
from transformers import AutoModel, AutoTokenizer, BertPreTrainedModel, BertModel, AdamW
from transformers import PreTrainedTokenizer, BertConfig, BertForTokenClassification
from typing import Dict, List, Union, Optional, Tuple
from tqdm import tqdm, trange
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
import numpy as np
from torch import nn
from torch.nn import CrossEntropyLoss
from torch.nn import functional as F

import torch
from torch.utils.data import DataLoader, Dataset

tokenizer = AutoTokenizer.from_pretrained("rlatmddus159/naval")

In [3]:
from datasets import load_dataset
from pprint import pprint

In [4]:
my_dataset = load_dataset('rlatmddus159/ex')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/rlatmddus159___json/rlatmddus159--ex-43f79b2686e066df/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from dataclasses import dataclass

@dataclass
class Config():
  model_name: str = "klue/bert-base"
  train_data = my_dataset['train']
  test_data = my_dataset['test']
  epoch: int = 10
  max_seq_len: int = 150
  batch_size: int = 16
  learning_rate: float = 5e-3
  adam_epsilon: float = 1e-8
  device: str = "cuda"
  max_grad_norm: float = 1.0
  seed: int = 1234
  intermediate_hidden_size: int = 768
  num_labels: int =  441

In [ ]:
init_config = Config()
init_config.train_data

Dataset({
    features: ['id', 'text', 'entities', 'relations', 'Comments', 'Articleid'],
    num_rows: 7934
})

In [ ]:
config = BertConfig.from_pretrained(init_config.model_name)
config.update(init_config.__dict__)

In [ ]:
model = BertForTokenClassification.from_pretrained(config.model_name, config=config)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['clas

In [ ]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
model.classifier = torch.nn.Linear(768, 441)

In [ ]:
max_length = init_config.max_seq_len
batch_size = init_config.batch_size

torch.manual_seed(init_config.seed)
np.random.seed(init_config.seed)

In [5]:
#라벨에 대한 리스트를 생성
#for data in dataset['train']['entities']
label_list=[]
for data in my_dataset['train']['entities']:
  for data2 in data:
    if data2['label'] in label_list:
      continue
    else:
      label_list.append(data2['label'])
for data in my_dataset['test']['entities']:
  for data2 in data:
    if data2['label'] in label_list:
      continue
    else:
      label_list.append(data2['label'])

In [ ]:
new_label_list=['0']
for i in label_list:
  new_label_list.append("B-"+i)
  new_label_list.append("I-"+i)
labels=new_label_list

In [7]:
len(label_list)

205

In [37]:
직위=['군사계급', '직위', '중령','원사',  '상사','준장','일병', '병장','중사','상병','하사','대위',  '대장','소령',  '중장','이등병', '중위', '대령', '소장','준위', '소위', '군무원',  '이병']
사람=['사람이름', '군인']
조직=['군사조직','기타조직', '1함대', '1훈련전대','장보고급', '고준봉급', '충남급','헬리콥터','전투기','대구급','2함대','해상전투단', '윤영하급','도산안창호급', '화생방지원대', '108조기경보전대','손원일급','군악대','해군본부','구조작전중대','1함대시설대대', '정조대왕급','세종대왕급','3특전대대', '135고속정전대', '전대', '군사경찰대대','의무대','118조기경보전대', 'AW-159', 'Lynx', '해군', '1무기지원대대', '솔개급','1수리창','동해합동작전지휘소','241해상전탐감시대','헌병대','훈련전대', '1지휘통신대대', '11전투전대', '1보급지원대대', '6구조작전중대','수송기','1공병대','해상전탐감시대', '헌병대대', '독도급','광개토대왕급','811유도탄기지','잠수함', '양양급', '강경급', '참수리급_신형','3함대', '1해상전투단', '충무공이순신급','기지전대','천지급','소양급','울산급', '인천급', '천왕봉급','조직', '참수리급', '해군조직', '1함대항공대', '함대사령부','1기지방호전대','무기지원대대','246해상전탐감지대', '근무지원대','육상훈련대','137고속정전대', '5분전투대기전대', '특전대대', '조기경보대대', '1기지지원대대', '1함대수송부대', 'UH-60','P-3','1stUDT', '249해상전탐감시대', '해군특수전단', '진해기지사령부','비행대대', '정비대대', '해병대', '포항급', '공병대대', '133고속정전대','군함', '12전투전대', '13전투전대', '지휘통신대대', '보급지원대대', '남포급', '군수전대',]
탑재체계=['채프','기타', '수중탄약','40mm','미사일','레이더', '대지', 'Dagaie','대잠', '76mm','R-BOC', '127mm', 'CIWS','20mm', '플레어','엔진', '대공','대함', '탱크','기관','수중유도무기', 'Mass']
특성=['순항속력', '배수톤수','최대속력', '길이', '높이','승조원', '폭','최대사거리','최고속도']
기타=['능력','기능','시스템','시뮬레이터', '훈련','영상감시장비','로켓포','기술', '로봇', '자주박격포','탄약운반장갑차','보병전투차', '스마트분배','소총','수직이착륙무인기','센서','무기','원격제어','지뢰제거장비','잠수정','연구과제', '측지장비','예산', '계획','마일즈장비','전자전지원수단기술', '기간', '가상현실', '감시장비','대포', '지역', '드론','군사목적','비행','탄창', '사격지휘차량', '영상센서', '웨어러블', '협약', '와이퍼', '착용구', '증강현실', '위성', '전지', '포병탐지레이더', '이착륙', '카메라', '헬멧', '선형폭약', '안전점검팀', '궤도차량']


In [53]:
import json

with open('/content/drive/MyDrive/모델만들기/dataset_change/해군_domain.jsonl', mode='r', encoding='utf-8') as f:
    with open('/content/drive/MyDrive/모델만들기/dataset_change/new_dataset.jsonl', mode='w', encoding='utf-8') as d:

        for entities in f.readlines():
            new_json = json.loads(entities)  # JSON 문자열을 파이썬 객체로 변환 / 한줄
            for i in range(len(new_json["entities"])):  # entities
                label = new_json["entities"][i]["label"]
                if label in 직위:
                    new_json["entities"][i]["label"] = "직위"
                elif label in 사람:
                    new_json["entities"][i]["label"] = "사람"
                elif label in 조직:
                    new_json["entities"][i]["label"] = "조직"
                elif label in 탑재체계:
                    new_json["entities"][i]["label"] = "탑재체계"
                elif label in 특성:
                    new_json["entities"][i]["label"] = "특성"
                elif label in 기타:
                    new_json["entities"][i]["label"] = "기타"

            json.dump(new_json, d, ensure_ascii=False)
            d.write('\n')  # 줄바꿈 추가



In [ ]:
def 전처리(dataset):
    line = []

    for data in dataset:
        start_offset = []  # (4,19,23)
        end_offset = []  # (7,22,25)
        entities_num = len(data['entities'])  # 3개
        text_len = len(data['text'])  # 30
        label2 = []
        text_made="##"+data['text']+ '\n'

        for entity in data['entities']:
            start_offset.append(entity['start_offset'])
            end_offset.append(entity['end_offset'])
            index_num_B = label_list.index(entity['label']) * 2 + 1
            label2.append(labels[index_num_B])
            index_num_I = label_list.index(entity['label']) * 2 + 2
            label2.append(labels[index_num_I])

        for text_inx in range(text_len):
            label_value = '0'
            for i in range(entities_num):
                if start_offset[i] == text_inx:
                    label_value = str(label2[2*i])
                    break
                elif start_offset[i] < text_inx < end_offset[i]:
                    label_value = str(label2[2*i+1])
                    break

            text_made = text_made + data['text'][text_inx] + '\t' + label_value + '\n'

        text_made = text_made.rstrip('\n')
        line.append(text_made)


    return line

In [ ]:
def load_data(document_list, tokenizer: PreTrainedTokenizer = None):
    data_list = []
    for doc in document_list:
        char_labels = []
        token_labels = []
        chars = []
        sentence = ""
        for line in doc.split("\n"):
            if line.startswith("##"):
                continue

            token, tag = line.split("\t")
            sentence += token
            char_labels.append(tag)
            chars.append(token)

        offset_mappings = tokenizer(sentence, max_length=max_length, return_offsets_mapping=True, truncation=True)["offset_mapping"]
        for offset in offset_mappings:
            start, end = offset
            if start == end == 0:
                continue
            token_labels.append(char_labels[start])

        instance = {
            "sentence": sentence,
            "token_label": token_labels,
            "char_label": char_labels,
            "offset_mapping": offset_mappings
        }
        data_list.append(instance)

    return data_list

In [ ]:
class NerDataset(Dataset):
    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        examples: List,
        shuffle: bool = False,
        **kwargs
    ):
        self.dataset = examples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        instance = self.dataset[index]

        return instance

In [ ]:
def collate_fn(input_examples):
    input_texts, input_labels_str = [], []
    offset_mappings = []
    char_labels = []
    for input_example in input_examples:
        text, label_strs = input_example["sentence"], input_example["token_label"]
        input_texts.append(text)
        input_labels_str.append(label_strs)
        offset_mappings.append(input_example["offset_mapping"])
        char_labels.append(input_example["char_label"])

    encoded_texts = tokenizer.batch_encode_plus(
        input_texts,
        add_special_tokens=True,
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
        return_token_type_ids=True,
        return_attention_mask=True,
        return_offsets_mapping=True
    )
    input_ids = encoded_texts["input_ids"]
    token_type_ids = encoded_texts["token_type_ids"]
    attention_mask = encoded_texts["attention_mask"]

    len_input = input_ids.size(1)
    input_labels = []
    for input_label_str in input_labels_str:
        input_label = [label2id[x] for x in input_label_str]
        if len(input_label) > max_length - 2:
            input_label = input_label[:max_length - 2]
            input_label = [-100] + input_label + [-100]
        else:
            input_label = (
                [-100] + input_label + (max_length - len(input_label_str) - 1) * [-100]
            )
        input_label = torch.tensor(input_label).long()
        input_labels.append(input_label)

    input_labels = torch.stack(input_labels)
    return input_ids, token_type_ids, attention_mask, input_labels, offset_mappings, char_labels

In [ ]:
type(전처리(init_config.train_data))

list

In [ ]:
examples = load_data(전처리(init_config.train_data), tokenizer)
index = int(len(examples) * 0.1)

In [ ]:
train_dataset = NerDataset(
    tokenizer,
    examples[index:]
)

In [ ]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)

In [ ]:
valid_dataset = NerDataset(
    tokenizer,
    examples[:index]
)

In [ ]:
valid_dataloader = DataLoader(
    dataset=valid_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn
)

In [ ]:
from transformers import AdamW

optimizer_grouped_parameters = [
    {'params': model.bert.parameters(), 'lr': config.learning_rate / 100 },
    {'params': model.classifier.parameters(), 'lr': config.learning_rate }
]
optimizer = AdamW(optimizer_grouped_parameters, lr=config.learning_rate, eps=config.adam_epsilon)

In [ ]:
def train_epoch(epoch, model, dataloader, optimizer):
    model.train()
    total_loss = 0.0

    tepoch = tqdm(dataloader, unit="batch", position=0, leave=True)
    for batch in tepoch:
        tepoch.set_description(f"Train")
        model.zero_grad()

        input_ids = batch[0].to(config.device)
        token_type_ids = batch[1].to(config.device)
        attention_mask = batch[2].to(config.device)
        labels = batch[3].to(config.device)

        inputs = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
            "labels": labels,
        }

        outputs = model(**inputs)

        loss = outputs[0]
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
        optimizer.step()
        total_loss += loss.item()

        tepoch.set_postfix(loss=loss.mean().item())
    tepoch.set_postfix(loss=total_loss / len(dataloader))
    return total_loss / len(dataloader)

In [ ]:
def valid_epoch(epoch, dataloader, model, tokenizer):
    total_loss = 0.0

    model.eval()
    all_char_preds = []
    all_char_labels = []
    all_token_predictions = []
    all_token_labels = []

    tepoch = tqdm(dataloader, unit="batch", position=0,leave=False)
    for batch in tepoch:
        tepoch.set_description(f"Valid")
        with torch.no_grad():
            input_ids = batch[0].to(config.device)
            token_type_ids = batch[1].to(config.device)
            attention_mask = batch[2].to(config.device)
            labels = batch[3].to(config.device)
            offset_mappings = batch[4]
            char_labels = batch[5]
            inputs = {
                "input_ids": input_ids,
                "token_type_ids": token_type_ids,
                "attention_mask": attention_mask,
                "labels": labels,
            }

            outputs = model(**inputs)

            loss, logits = outputs[:2]
            total_loss += loss.item()

            token_predictions = logits.argmax(dim=2) # logits
            token_predictions = token_predictions.detach().cpu().numpy()

            char_predictions = token_to_char_label(token_predictions, labels, offset_mappings)
            for j, (char_pred, char_label) in enumerate(zip(char_predictions, char_labels)):
                if len(char_pred) != len(char_label): # unknown 문장 처리
                    del char_predictions[j]
                    del char_labels[j]

            all_char_preds.extend(char_predictions)
            all_char_labels.extend(char_labels)

            for token_prediction, label in zip(token_predictions, labels):
                filtered = []
                filtered_label = []
                for i in range(len(token_prediction)):
                    if label[i].tolist() == -100:
                        continue
                    filtered.append(id2label[token_prediction[i]])
                    filtered_label.append(id2label[label[i].tolist()])
                assert len(filtered) == len(filtered_label)
                all_token_predictions.append(filtered)
                all_token_labels.append(filtered_label)

        tepoch.set_postfix(loss=loss.mean().item())

    token_f1 = f1_score(all_token_labels, all_token_predictions, average="macro")
    return total_loss / len(dataloader),  token_f1

In [ ]:
def token_to_char_label(token_predictions, labels, offset_mapping_batch):
    char_predictions = []
    for token_predicts, label, offset_mappings in zip(token_predictions, labels, offset_mapping_batch):

        # SPECIAL token 제외
        filtered = []
        for i in range(len(token_predicts)):
            if label[i].tolist() == -100:
                continue
            filtered.append(token_predicts[i])
        char_prediction = []

        # SPECIAL token 제외
        if offset_mappings[0][0] == 0 and offset_mappings[0][1] == 0:
            del offset_mappings[0]
        if offset_mappings[-1][0] == 0 and offset_mappings[-1][1] == 0:
            del offset_mappings[-1]
        assert len(filtered) == len(offset_mappings)

        prev_end = None
        for token_predict, offset_mapping in zip(filtered, offset_mappings):
            start, end = offset_mapping

            # 이전 end와 현재 start가 1개이상 차이나면 띄어쓰기를 추가한다
            if prev_end != None and start - prev_end > 0:
                char_prediction.append("O") # 띄어쓰기
            prev_end = end

            # 싱글 라벨
            if end - start == 1:
                label_str = id2label[token_predict]
                char_prediction.append(label_str)
                continue

            # 멀티 라벨
            for i in range(end - start):
                label_str = id2label[token_predict]
                if i == 0 or label_str == "0":
                    char_prediction.append(label_str)
                    continue
                char_prediction.append("I-" + label_str.split("-")[1])
        char_predictions.append(char_prediction)
    return char_predictions

In [ ]:
def test_epoch(dataloader, model, tokenizer):
    total_loss = 0.0

    model.eval()
    all_char_preds = []
    all_char_labels = []
    all_token_predictions = []
    all_token_labels = []

    tepoch = tqdm(dataloader, unit="batch")
    for batch in tepoch:
        tepoch.set_description(f"Test")
        with torch.no_grad():

            input_ids = batch[0].to(config.device)
            token_type_ids = batch[1].to(config.device)
            attention_mask = batch[2].to(config.device)
            labels = batch[3].to(config.device)
            offset_mappings = batch[4]
            char_labels = batch[5]

            inputs = {
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "token_type_ids": token_type_ids,
                "labels": labels,
            }

            outputs = model(**inputs)

            loss, logits = outputs[:2]
            total_loss += loss.item()

            token_predictions = logits.argmax(dim=2) # logits
            token_predictions = token_predictions.detach().cpu().numpy()

            char_predictions = token_to_char_label(token_predictions, labels, offset_mappings)
            for j, (char_pred, char_label) in enumerate(zip(char_predictions, char_labels)):
                if len(char_pred) != len(char_label):
                    # print(tokenizer.decode(batch[0][j]))
                    del char_predictions[j]
                    del char_labels[j]

            all_char_preds.extend(char_predictions)
            all_char_labels.extend(char_labels)

            for token_prediction, label in zip(token_predictions, labels):
                filtered = []
                filtered_label = []
                for i in range(len(token_prediction)):
                    if label[i].tolist() == -100:
                        continue
                    filtered.append(id2label[token_prediction[i]])
                    filtered_label.append(id2label[label[i].tolist()])
                assert len(filtered) == len(filtered_label)
                all_token_predictions.append(filtered)
                all_token_labels.append(filtered_label)

            tepoch.set_postfix(loss=loss.mean().item())

    token_result = classification_report(all_token_labels, all_token_predictions)
    token_f1 = f1_score(all_token_labels, all_token_predictions, average="macro")
    char_result = classification_report(all_char_labels, all_char_preds)
    char_f1 = f1_score(all_char_labels, all_char_preds)

    print(token_result)
    print(char_result)

    tepoch.set_postfix(loss=total_loss / len(dataloader), token_f1=token_f1, char_f1=char_f1)
    return total_loss / len(dataloader), token_f1, char_f1

In [ ]:
examples2 = load_data(전처리(init_config.test_data), tokenizer)

In [ ]:
test_dataset = NerDataset(
    tokenizer,
    examples2,
)

In [ ]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn
)

In [ ]:
# String label값을 tensor로 변환하기 위해
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

In [ ]:
model.to("cuda")

best_f1 = 0.0
best_model = None

tepoch = trange(config.epoch, position=0, leave=True)
for epoch in tepoch:
    tepoch.set_description(f"Epoch {epoch}")

    train_loss = train_epoch(epoch, model, train_dataloader, optimizer)
    valid_loss, token_f1 = valid_epoch(epoch, valid_dataloader, model, tokenizer)

    if best_f1 < token_f1:
        best_f1 = token_f1
        best_model = model

    tepoch.set_postfix(valid_f1=token_f1)
    print(f'train_loss: {train_loss:.4f}, valid_loss: {valid_loss:.4f}, token_f1: {token_f1:.4f}')
    print('=' * 100)
test_loss, token_f1, char_f1 = test_epoch(test_dataloader, model, tokenizer)

Epoch 1:  10%|█         | 1/10 [03:20<30:01, 200.15s/it, valid_f1=0.382]

train_loss: 0.6272, valid_loss: 0.5292, token_f1: 0.3823


Epoch 2:  20%|██        | 2/10 [06:41<26:47, 200.96s/it, valid_f1=0.492]

train_loss: 0.3552, valid_loss: 0.4725, token_f1: 0.4915


Epoch 3:  30%|███       | 3/10 [10:01<23:22, 200.40s/it, valid_f1=0.505]

...

train_loss: 0.0381, valid_loss: 0.8606, token_f1: 0.4967


Epoch 9: 100%|██████████| 10/10 [33:19<00:00, 199.92s/it, valid_f1=0.538]


train_loss: 0.0362, valid_loss: 0.9008, token_f1: 0.5375


Test: 100%|██████████| 56/56 [00:13<00:00,  4.24batch/s, loss=0.528]
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

   108조기경보전대       0.50      1.00      0.67         1
   118조기경보전대       1.00      1.00      1.00         2
      11전투전대       0.33      0.50      0.40         2
       127mm       1.00      1.00      1.00         2
      12전투전대       0.00      0.00      0.00         1
    135고속정전대       0.00      0.00      0.00         1
    137고속정전대       0.00      0.00      0.00         1
     1기지방호전대       0.00      0.00      0.00         1
     1무기지원대대       0.33      1.00      0.50         1
     1보급지원대대       0.00      0.00      0.00         1
        1수리창       0.50      1.00      0.67         1
     1지휘통신대대       0.00      0.00      0.00         0
         1함대       0.86      0.91      0.88        55
     1함대시설대대       0.00      0.00      0.00         0
      1해상전투단       0.80      1.00      0.89         4
       1훈련전대       0.00      0.00      0.00         0
        20mm       1.00      1.00      1.00         3
  241해상전탐감시대       1.00    

In [ ]:
model.save_pretrained("/content/drive/MyDrive/모델만들기/ner결과물4")
tokenizer.save_pretrained("/content/drive/MyDrive/모델만들기/ner결과물4")

('/content/drive/MyDrive/모델만들기/ner결과물4/tokenizer_config.json',
 '/content/drive/MyDrive/모델만들기/ner결과물4/special_tokens_map.json',
 '/content/drive/MyDrive/모델만들기/ner결과물4/vocab.txt',
 '/content/drive/MyDrive/모델만들기/ner결과물4/added_tokens.json',
 '/content/drive/MyDrive/모델만들기/ner결과물4/tokenizer.json')